# NE551 Stopping Power Assignment #

## Using the stopping power equations for *heavy ions*, develop a code that will: ##

## Calculate the collisional mass stopping power of a charged particle (heavy ions with a given charge, Z) and kinetic energy through the materials listed below. For this assignment, heavy ions are any charged particle with mass greater than or equal to the proton ##

In [21]:
%%javascript
MathJax.Extension["TeX/cancel"]={version:"2.4.0",ALLOWED:{color:1,mathcolor:1,background:1,mathbackground:1,padding:1,thickness:1}};MathJax.Hub.Register.StartupHook("TeX Jax Ready",function(){var c=MathJax.InputJax.TeX,a=MathJax.ElementJax.mml,b=MathJax.Extension["TeX/cancel"];b.setAttributes=function(h,e){if(e!==""){e=e.replace(/ /g,"").split(/,/);for(var g=0,d=e.length;g<d;g++){var f=e[g].split(/[:=]/);if(b.ALLOWED[f[0]]){if(f[1]==="true"){f[1]=true}if(f[1]==="false"){f[1]=false}h[f[0]]=f[1]}}}return h};c.Definitions.Add({macros:{cancel:["Cancel",a.NOTATION.UPDIAGONALSTRIKE],bcancel:["Cancel",a.NOTATION.DOWNDIAGONALSTRIKE],xcancel:["Cancel",a.NOTATION.UPDIAGONALSTRIKE+" "+a.NOTATION.DOWNDIAGONALSTRIKE],cancelto:"CancelTo"}},null,true);c.Parse.Augment({Cancel:function(e,g){var d=this.GetBrackets(e,""),f=this.ParseArg(e);var h=b.setAttributes({notation:g},d);this.Push(a.menclose(f).With(h))},CancelTo:function(e,g){var i=this.ParseArg(e),d=this.GetBrackets(e,""),f=this.ParseArg(e);var h=b.setAttributes({notation:a.NOTATION.UPDIAGONALSTRIKE+" "+a.NOTATION.UPDIAGONALARROW},d);i=a.mpadded(i).With({depth:"-.1em",height:"+.1em",voffset:".1em"});this.Push(a.msup(a.menclose(f).With(h),i))}});MathJax.Hub.Startup.signal.Post("TeX cancel Ready")});MathJax.Ajax.loadComplete("[MathJax]/extensions/TeX/cancel.js");

<IPython.core.display.Javascript object>

In [22]:
import pyne.data as data
import math
import scipy.constants as const

Equation 2.22 from Anderson: 

\begin{equation}
S_c = 4 z^2 \pi r_o^2 m_e \cancel{c^2} N_A \frac{Z}{M_m} \rho \frac{\left( T + M \cancel{c^2} \right) ^2}{T \left( T + 2 M \cancel{c^2} \right) } ln \left( \frac{2 m_e \cancel{c^2} T \left( T + 2 M \cancel{c^2} \right) }{I \left( T + M \cancel{c^2} \right)^2} \right)
\end{equation}

First, import useful constants:

In [23]:
pi = math.pi
r = const.value('classical electron radius') * 100.0 # Convert 1m = 100cm
m_e = const.value('electron mass energy equivalent in MeV')
N_A = const.value('Avogadro constant')
m_to_e = const.physical_constants['atomic mass constant energy equivalent in MeV'][0]

In [24]:
def I(material):
    """Looks up the Adjusted Average Excitation Energy if Z<=13. Otherwise calculates it"""
    # For now, specify material with Z value
    Z = material
    
    # Z must be an integer
    assert type(Z) == type(4)
    
    # Lookup table for Z<=13. Key is the "Z" of the material. Values are in eV
    lookup_table = {
        1: 18.9,
        2: 42.0,
        3: 38.0,
        4: 60.0, 
        6: 78.0,
        7: 85.0,
        8: 89.0,
        10: 131.0,
        13: 163.0
    }
    
    # Check to see if Z is in our table
    I_a = lookup_table.get(Z)
    
    # If I is not in the table, calculate it
    # Use Anderson Equation 2.33
    if I_a is None:
        I_a = 9.73 * Z + 58.8 * Z ** -0.19
    
    return I_a

In [ ]:
def beta_squared(T, m):
    """ Gives value of beta^2 for a given Mass (MeV/c^2) and Kinetic Energy (MeV)
    """
    numerator = T * (T + 2 * m)
    denominator = (T + M) ** 2
    return numerator / denominator

In [28]:
def S_c(incident, target, T, density, M_m, M_b):
    """ Returns the stopping power in MeV/cm
        T in MeV
        density in g/cm^3
    """
    # Currently the incident and target are specified in Z number. incident is assumed to be fully ionized
    z = incident
    Z = target
    
    # Convert the mass of the beam particles into MeV
    M_b = M_b * m_to_e
    
    first = 4 * (z ** 2) * pi * (r ** 2) * m_e
    second = N_A * Z * density / M_m # TODO: Take M_m from a Pyne material
    third = (T + M_b) ** 2 / (T * (T + 2 * M_b))
    logpart = (2 * m_e * T * (T + 2 * M_b)) / (I(Z) * (T + M_b) ** 2)
    fourth = math.log(logpart)
    
    print T
    print m_e
    print M_b
    print I(Z)
    
    return first * second * third * fourth

In [29]:
S_c(14, 13, 500, 2.7, 26.98154, 28.0)

500
0.5109989461
26081.8346712
163.0


-17568.0085564914